# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[2]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 625


,peak_channel_id,waveform_halfwidth,waveform_amplitude,waveform_velocity_below,L_ratio,snr,amplitude_cutoff,waveform_spread,nn_miss_rate,waveform_repolarization_slope,presence_ratio,isi_violations,nn_hit_rate,local_index_unit,d_prime,max_drift,waveform_PT_ratio,cluster_id,isolation_distance,silhouette_score,waveform_velocity_above,cumulative_drift,firing_rate,waveform_duration,waveform_recovery_slope,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951808989,850211269,0.164824,100.057815,0.000000,0.036386,2.840270,0.026085,30.0,0.034483,0.413804,0.99,0.024853,0.938000,5,3.335647,16.27,0.346582,5,95.979386,0.055628,0.000000,137.52,31.839506,0.302178,-0.071154,NaN,1000.0,1.652282,0.997110,2.086087,0.890021,2.062046,0.193774,0.033770,0.016799,0.008807,40.000,0.633406,0.000,0.08,1.0,NaN,NaN,-0.229678,NaN,-0.195122,225.0,0.0,NaN,NaN,0.187761,NaN,0.694848,-15.000,4911,0.50,35.170606,37.595905,28.003696,31.890527,30.338949,NaN,0.1795,False,False,False,False,False,0.199017,False,0.004952,0.027629,0.006559,0.013269,0.006948,140,11,757968489,13,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951808976,850211269,0.164824,120.798210,0.000000,0.070177,3.305557,0.043860,20.0,0.030250,0.546024,0.99,0.008103,0.904667,4,3.853885,13.74,0.684731,4,69.787442,0.204123,NaN,200.04,28.346921,0.260972,-0.153623,NaN,800.0,3.576891,0.909264,1.623596,1.236158,1.414272,0.105344,0.020643,0.011077,0.012017,48.750,1.310929,0.000,0.02,15.0,NaN,NaN,0.170188,NaN,-0.002551,270.0,0.0,NaN,NaN,0.084907,NaN,0.993035,-18.750,5022,0.75,32.582967,23.420837,30.048144,9.436456,29.782766,NaN,0.1525,False,False,False,False,False,0.001627,False,0.004001,0.005228,0.003936,0.054303,0.003511,140,11,757968489,13,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951809003,850211277,0.137353,76.722945,1.030151,0.006293,1.926873,0.023672,70.0,0.000067,0.315857,0.98,0.000000,0.769231,6,3.862677,19.92,0.189503,6,41.314878,NaN,0.343384,310.35,0.279998,0.233501,-0.019571,NaN,4100.0,0.812121,NaN,1.291429,0.800000,1.180000,1.841297,0.100145,0.169811,0.037816,52.195,0.446237,0.344,0.32,8.0,NaN,NaN,0.492308,-1.0,0.590909,315.0,120.0,NaN,NaN,0.341355,0.082097,0.130740,-9.756,4920,0.75,0.170497,0.000000,0.498351,0.129387,0.454938,NaN,0.1395,False,False,False,False,False,0.194797,True,0.191987,NaN,0.157744,0.680890,0.172531,180,11,757968489,17,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.964361,1249.998515,True
951809147,850211317,0.123618,122.473455,-1.373534,0.003246,3.021903,0.003017,60.0,0.000553,0.513822,0.99,0.015365,0.985333,14,4.807625,19.06,0.527284,14,71.310975,0.116839,-0.137353,188.33,9.868341,0.260972,-0.116531,NaN,500.0,4.663087,0.981026,1.146081,1.042857,2.670968,0.363796,0.051481,0.019883,0.050015,14.000,1.007488,0.000,0.08,1.0,NaN,NaN,0.423645,NaN,0.416667,45.0,30.0,NaN,NaN,0.000116,NaN,0.040909,2.000,5001,0.25,7.448927,8.952675,17.706234,4.205067,13.782680,NaN,0.1575,False,False,Fals

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 10


,peak_channel_id,waveform_halfwidth,waveform_amplitude,waveform_velocity_below,L_ratio,snr,amplitude_cutoff,waveform_spread,nn_miss_rate,waveform_repolarization_slope,presence_ratio,isi_violations,nn_hit_rate,local_index_unit,d_prime,max_drift,waveform_PT_ratio,cluster_id,isolation_distance,silhouette_score,waveform_velocity_above,cumulative_drift,firing_rate,waveform_duration,waveform_recovery_slope,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951816073,850213733,0.178559,103.167285,-0.686767,0.000006,3.564781,0.011487,30.0,0.000000,0.454228,0.98,0.032281,1.000000,316,6.501919,20.74,2.450593,324,121.599760,0.173476,2.747069,285.71,4.614934,0.274707,-0.069146,NaN,500.0,16.416878,1.253333,3.508022,1.466667,2.880675,0.972518,0.124576,0.142769,0.142939,28.0,0.381998,0.006,0.32,2.0,NaN,NaN,0.553588,NaN,0.488966,315.0,0.0,NaN,NaN,8.869927e-04,NaN,0.070696,38.0,4979,0.75,1.216249,0.186514,10.547309,0.043860,9.626961,NaN,0.0785,False,False,False,False,False,0.450610,False,0.233520,0.675676,0.191346,0.886068,0.101289,2240,59,757968493,222,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.992397,1249.999683,True
951816053,850213733,0.357119,42.876990,0.529792,0.017613,1.536168,0.033473,110.0,0.066667,0.070159,0.99,0.079527,0.954000,314,3.053332,65.24,0.695381,322,135.360822,0.082734,2.668581,187.55,8.254896,0.865327,-0.016053,NaN,1500.0,4.273203,1.336371,1.854505,2.123810,1.598958,0.380156,0.026172,0.010104,0.028133,66.0,0.434865,0.000,0.32,15.0,NaN,NaN,-0.278125,-0.881818,-0.736000,135.0,120.0,NaN,NaN,3.872754e-01,0.003902,0.010511,-22.0,4945,0.00,8.430610,7.753656,6.854676,8.342151,7.174427,NaN,0.1295,False,False,False,False,False,0.366424,False,0.030664,0.014653,0.103644,0.073143,0.049589,2240,59,757968493,222,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.992397,1249.999683,True
951816231,850213757,0.151089,82.650750,-0.206030,0.011925,2.240731,0.009710,90.0,0.009374,0.389823,0.99,0.084126,0.980000,332,4.102027,50.52,0.517280,340,69.653279,0.074741,0.196219,133.23,8.906172,0.247236,-0.085636,NaN,800.0,7.819476,1.474929,1.796552,2.474510,1.756807,0.379288,0.044417,0.203975,0.244105,25.0,0.358231,0.000,0.02,15.0,NaN,NaN,0.371622,0.471572,0.311594,225.0,90.0,NaN,NaN,7.270743e-02,0.083094,0.000001,20.0,4922,0.25,9.493834,21.076088,7.152209,6.878327,11.246886,NaN,0.1065,False,False,False,False,False,0.082898,False,0.092352,0.000029,0.048192,0.278723,0.069536,2360,59,757968493,234,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.992397,1249.999683,True
951816279,850213765,0.164824,60.959145,-0.755444,0.014219,1.900495,0.025669,100.0,0.027717,0.238303,0.99,0.045408,0.937333,338,3.723697,58.47,0.517128,346,76.265823,0.074256,-0.073582,145.97,9.342515,0.370854,-0.040695,NaN,800.0,3.638293,1.141469,1.516279,0.602299,1.653307,0.426543,0.051364,0.028790,0.129477,27.5,1.842614,0.000,0.04,2.0,NaN,NaN,0.298035,-0.155844,0.185606,45.0,30.0,NaN,NaN,1.326845e-02,0.517142,0.342429,22.5,4993,0.25,12.1

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951816073     9.626961
951816053     7.174427
951816231    11.246886
951816279    11.809730
951816287     8.914914
951816326     9.329886
951816343     8.247494
951816369     9.984650
951816394     0.470258
951816426     5.937503
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

8.274270864266825


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951816073     1.216249
951816053     8.430610
951816231     9.493834
951816279    12.163545
951816287     4.788218
951816326     8.162156
951816343     5.618465
951816369     9.761759
951816394     0.525258
951816426     6.495308
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

6.665540150944315


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])